In [1]:
from pycalphad import Database, Model
from pycalphad.core.utils import make_callable
import pycalphad.variables as v
import numba
import numpy as np

In [2]:
dbf = Database('Al-Ni/Al-Ni-Dupin-2001.tdb')
mod = Model(dbf, ['AL', 'NI', 'VA'], 'FCC_L12')

In [3]:
from sympy import lambdify
from pycalphad.core.utils import NumPyPrinter
@numba.vectorize
def where(condition, x, y):
    if condition:
        return x
    else:
        return y
func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules=[{'where': where}, 'numpy'], printer=NumPyPrinter)
pure_func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules=['numpy'], printer=NumPyPrinter)

In [4]:
vec_func = numba.vectorize(['float64({})'.format(','.join(['float64'] * len(tuple(mod.variables))))], nopython=True, target='parallel')(func)


x1 = np.random.rand(5, 1, 1)*2000
x2 = np.random.rand(5, 5, 3000)
x3 = np.random.rand(5, 5, 3000)
x4 = np.random.rand(5, 5, 3000)
x5 = np.random.rand(5, 5, 3000)
x6 = np.random.rand(5, 5, 3000)
np.testing.assert_allclose(vec_func(x1, x2, x3, x4, x5, x6), pure_func(x1, x2, x3, x4, x5, x6))
%timeit vec_func(x1, x2, x3, x4, x5, x6)

100 loops, best of 3: 13.8 ms per loop


In [82]:
import sympy
def make_gradient_from_graph(graph, wrt):
    dummies = [str(sympy.Dummy(str(i))) for i in range(len(wrt))]
    grads = np.empty((len(wrt)), dtype=object)
    hess = np.empty((len(wrt), len(wrt)), dtype=object)
    for i in range(len(wrt)):
        grads[i] = graph.diff(wrt[i])
        for j in range(i, len(wrt)):
            hess[i][j] = lambdify(tuple(wrt), grads[i].diff(wrt[j]), dummify=True,
                                         modules=[{'where': where}, 'numpy'], printer=NumPyPrinter)
        grads[i] = lambdify(tuple(wrt), grads[i], dummify=True,
                            modules=[{'where': where}, 'numpy'], printer=NumPyPrinter)
    grads = grads.tolist()
    hess = hess.tolist()
    print(grads)
    return
    gradstr = "def dyn_grad_func ({0},result): [result.__setitem__([Ellipsis, i], grads[i]({0})) for i in range(result.shape[-1])]"
    gradstr = gradstr.format(','.join(dummies))
    print(gradstr)
    namespace = {'grads': grads}
    lambda_grad_func = numba.jit(eval(gradstr, namespace), nopython=True)
    # numba needs well-defined arguments so we do this wrapping
    import itertools
    def grad_func(*args):
        result = np.zeros(args[-1].shape + (len(args),))
        lambda_grad_func(*itertools.chain(args, [result]))
        return result
    def hess_func(*args):
        result = np.empty(args[-1].shape + (len(args), len(args)))
        for i in range(result.shape[-1]):
            for j in range(i, result.shape[-1]):
                result[..., i, j] = result[..., j, i] = hess[i][j](*args)
        return result
    return grad_func, hess_func

In [83]:
grad_func, hess_func = make_gradient_from_graph(mod.ast, mod.variables)

[<function <lambda> at 0x7ff54a893488>, <function <lambda> at 0x7ff53d2bc938>, <function <lambda> at 0x7ff544281758>, <function <lambda> at 0x7ff54223f410>, <function <lambda> at 0x7ff5442818c0>, <function <lambda> at 0x7ff54ae34320>]


TypeError: 'NoneType' object is not iterable

In [ ]:
%time result = grad_func(x1, x2, x3, x4, x5, x6)
print(result)
#%time hess_func(x1, x2, x3, x4, x5, x6).shape

In [ ]:
%timeit pure_func(x1, x2, x3, x4, x5, x6)

In [62]:
arr = np.zeros((2,2))
arr.__setitem__([Ellipsis, 0], 1)
print(arr)

[[ 1.  0.]
 [ 1.  0.]]
